In [7]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader

llm = ChatOpenAI(
    temperature = 0.1
    , model="gpt-4o-mini-2024-07-18"
)


In [8]:

# Wikipedia URL 검색 도구
class WikipediaUrlSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="URL을 찾기위한 질의입니다."
    )

class WikipediaUrlSearchTool(BaseTool):
    name = "WikipediaUrlSearchTool"
    description = "질의를 받아 Wikipedia 검색 결과의 첫 번째 URL을 반환합니다."
    args_schema: Type[WikipediaUrlSearchToolArgsSchema] = WikipediaUrlSearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wiki.run(query)

In [9]:
class DuckDuckGoUrlSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="URL을 찾기위한 질의입니다."
    )

class DuckDuckGoUrlSearchTool(BaseTool):
    name = "DuckDuckGoUrlSearchTool"
    description = "질의를 받아 DuckDuckGo 검색 결과의 첫 번째 URL을 반환합니다."
    args_schema: Type[DuckDuckGoUrlSearchToolArgsSchema] = DuckDuckGoUrlSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchRun()
        return ddg.run(query)

In [10]:
# URL 내용 스크래핑 도구
class WebResearchToolArgsSchema(BaseModel):
    url: str = Field(
        description="스크래핑할 URL입니다."
    )

class WebResearchTool(BaseTool):
    name = "WebResearchTool"
    description = "URL에서 텍스트를 로드하여 문서로 반환합니다."
    args_schema: Type[WebResearchToolArgsSchema] = WebResearchToolArgsSchema

    def _run(self, url):
        wl = WebBaseLoader(url)
        return wl.load()

In [11]:
# 결과 파일 저장 도구
class SaveFileToolArgsSchema(BaseModel):
    doc: str

class SaveFileTool(BaseTool):
    name = "SaveFileTool"
    description = "스크래핑된 문서를 파일로 저장합니다."
    args_schema: Type[SaveFileToolArgsSchema] = SaveFileToolArgsSchema

    def _run(self, doc):
        with open('research_result.txt', 'w', encoding='utf-8') as f:
            f.write(doc)
        return "저장"

In [13]:
# 에이전트 초기화
agent = initialize_agent(
    llm = llm
    # , verbose = True # 안쓰면 파일 저장 안댐
    , agent = AgentType.OPENAI_FUNCTIONS
    , handle_parsing_errors = True
    , tools=[
        WikipediaUrlSearchTool()
        , DuckDuckGoUrlSearchTool()
        , WebResearchTool()
        , SaveFileTool()
    ]
)

# 에이전트 실행
prompt = "Research about the XZ backdoor."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaUrlSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously 

{'input': 'Research about the XZ backdoor.',
 'output': 'The XZ backdoor refers to a malicious backdoor that was introduced to the Linux utility xz, specifically within the liblzma library, in versions 5.6.0 and 5.6.1. This incident occurred in February 2024 and was attributed to an account using the name "Jia Tan". The backdoor allows an attacker who possesses a specific Ed448 private key to execute remote code on affected Linux systems.\n\nThis vulnerability has been assigned the Common Vulnerabilities and Exposures (CVE) number CVE-2024-3094 and has been given a CVSS score of 10.0, which is the highest possible score, indicating a critical security issue. At the time of its discovery, the backdoored version had not been widely deployed in production systems but was present in development versions of major Linux distributions.\n\nThe backdoor was discovered by software developer Andres Freund, who publicly announced his findings on March 29, 2024.\n\nFor more detailed information, yo